In [ ]:

import numpy as np
import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense, Embedding, Lambda
from tensorflow.python.keras import utils
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
import gensim


In [ ]:
data=open('CBOW.txt','r')
c_data = [text for text in data if text.count(' ') >= 2]

In [ ]:
vectorize = Tokenizer()
vectorize.fit_on_texts(c_data)

In [ ]:
# Calculate the total number of words in all sentences combined
total_vocab = sum(len(s) for s in c_data)

# Calculate the total number of unique words in the vocabulary
word_count = len(vectorize.word_index) + 1

print(total_vocab)
print(word_count)

1191
103


In [ ]:
window_size = 2
total_length = window_size * 2
embedding_dim = 100

# Function to generate CBOW data
def generate_cbow_data(data, window_size):
    for text in data:
        text_len = len(text)
        for idx, word in enumerate(text):
            context_word = [text[i] for i in range(idx - window_size, idx + window_size + 1) if 0 <= i < text_len and i != idx]
            target = [word]
            contextual = sequence.pad_sequences([context_word], maxlen=total_length)
            final_target = to_categorical(target, total_vocab)
            yield contextual, final_target

# CBOW model
cbow_model = Sequential()
cbow_model.add(Embedding(input_dim=total_vocab, output_dim=embedding_dim))
cbow_model.add(Lambda(lambda x: K.mean(x, axis=1)))
cbow_model.add(Dense(total_vocab, activation='softmax'))
cbow_model.compile(loss='categorical_crossentropy', optimizer='adam')

# Training the CBOW model
epochs = 10
for epoch in range(epochs):
    cost = 0
    for x, y in generate_cbow_data(data, window_size):
        cost += cbow_model.train_on_batch(x, y)
    print(f'Epoch {epoch + 1}, Loss: {cost}')

# Save word vectors
word_vectors = cbow_model.get_weights()[0]

with open('vectors.txt', 'w') as vect_file:
    vect_file.write(f'{total_vocab} {embedding_dim}\n')
    for word, i in vectorize.word_index.items():
        vect_file.write(f'{word} {" ".join(map(str, word_vectors[i, :]))}\n')

import gensim
cbow_output = gensim.models.KeyedVectors.load_word2vec_format('vectors.txt', binary=False, limit=100)

# Get similar words
cbow_output.most_similar(positive=['covid'])


Epoch 1, Loss: 0
Epoch 2, Loss: 0
Epoch 3, Loss: 0
Epoch 4, Loss: 0
Epoch 5, Loss: 0
Epoch 6, Loss: 0
Epoch 7, Loss: 0
Epoch 8, Loss: 0
Epoch 9, Loss: 0
Epoch 10, Loss: 0


[('who', 0.2629571557044983),
 ('48', 0.24375006556510925),
 ('this', 0.23177707195281982),
 ('infections', 0.205921933054924),
 ('is', 0.19043906033039093),
 ('does', 0.16252131760120392),
 ('illness', 0.16196690499782562),
 ('the', 0.14322620630264282),
 ('and', 0.14139805734157562),
 ('19', 0.14037524163722992)]